In [17]:
from PIL import Image
from torchvision import transforms
import torch

filename = "../data/minimalist-4k-wallpaper-full-hd.jpg"

In [18]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.eval()

Using cache found in /home/carles/.cache/torch/hub/pytorch_vision_v0.10.0


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [19]:
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')
    
with torch.no_grad():
    output = model(input_batch)


print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)


tensor([-0.7078,  0.6967,  1.2135,  1.3125,  0.4443,  0.9637,  1.0806, -0.3405,
        -0.1046, -0.3838, -0.1112,  0.6114,  0.0998,  0.3613, -0.5853, -0.2264,
        -0.2140, -0.7147, -0.7333, -0.4465, -0.4423, -0.6094,  0.8113, -0.3930,
        -1.3984, -0.2488, -0.8072, -0.9216, -0.9285, -0.9787, -0.7711, -0.7976,
        -0.8260, -0.0283,  0.4088, -1.2062, -0.9969, -1.5558, -1.5875, -0.0104,
         0.2832, -1.2856, -0.2541,  0.3487, -0.3434, -0.4405,  0.1212, -1.0697,
        -0.5487, -0.8268,  0.6263,  0.1770, -0.7100, -0.8520, -1.0458, -0.4479,
        -0.7004, -0.4922, -0.8023, -0.0535, -0.4192, -1.3397, -0.7133, -0.6056,
         0.0491, -0.5960, -0.3682, -0.3253, -0.6222, -0.1542, -0.4384, -0.4607,
         0.0558, -0.8891, -0.6367, -0.0279,  0.1373,  0.0887, -0.9705, -1.0338,
         0.8623,  0.3605, -0.5760,  0.7822, -1.3681, -0.6341, -0.3643, -1.6855,
        -0.4352, -1.3453, -0.2808, -0.7258, -0.0583, -2.2511, -1.4156, -0.4711,
        -1.2667, -1.5855, -1.6108, -0.71

In [20]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt


--2022-05-17 11:39:18--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.1’

imagenet_classes.tx 100%[===================>]  10,23K  --.-KB/s    in 0s      

2022-05-17 11:39:19 (50,0 MB/s) - ‘imagenet_classes.txt.1’ saved [10472/10472]



In [21]:
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

alp 0.606482207775116
valley 0.16636939346790314
mountain tent 0.048262834548950195
geyser 0.03098953701555729
volcano 0.022045640274882317
